In [20]:
!pip install --upgrade pip
!pip install opencv-python==3.3.0.10 opencv-contrib-python==3.3.0.10

ERROR: Could not find a version that satisfies the requirement opencv-python==3.3.0.10 (from versions: 3.4.0.14, 3.4.8.29, 3.4.9.31, 3.4.9.33, 3.4.10.35, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.14.51, 3.4.14.53, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 4.1.2.30, 4.2.0.32, 4.2.0.34, 4.3.0.36, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.2.52, 4.5.2.54, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62)
ERROR: No matching distribution found for opencv-python==3.3.0.10


In [15]:
!pip install tensorflow

In [19]:

import cv2 
import matplotlib.pyplot as plt
import os
from pathlib import Path
import numpy as np 
import pandas as pd 
%matplotlib inline

import os 

path = './car_models_database_train/'
carModels = os.listdir(path)
trainPath = '../datasets/datasets_train/train/'
annotation_file = "../datasets/datasets_train/train_annotation/_annotation.csv"
#reading image
img1 = cv2.imread(path + 'AcuraZDXHatchback2012/1.jpg')  
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

#keypoints
sift = cv2.xfeatures2d.SIFT_create()
keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)

img_1 = cv2.drawKeypoints(gray1,keypoints_1,img1)
plt.imshow(img_1)

AttributeError: module 'cv2' has no attribute 'xfeatures2d'

In [ ]:

# read images
img1 = cv2.imread(path + 'AcuraZDXHatchback2012/1.jpg')  
img2 = cv2.imread(path + 'AcuraZDXHatchback2012/4.jpg') 
img3 = cv2.imread(path + 'BMW1SeriesConvertible2012/1.jpg') 
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

figure, ax = plt.subplots(1, 2, figsize=(16, 8))

ax[0].imshow(img1, cmap='gray')
ax[1].imshow(img2, cmap='gray')

In [10]:
keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
keypoints_3, descriptors_3 = sift.detectAndCompute(img3,None)
#feature matching
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

matches = bf.match(descriptors_1,descriptors_2)
matches = sorted(matches, key = lambda x:x.distance)

img4 = cv2.drawMatches(img1, keypoints_1, img2, keypoints_2, matches[:], img2, flags=2)
plt.imshow(img4),plt.show()

NameError: name 'sift' is not defined

In [11]:
matches = bf.match(descriptors_1,descriptors_3)
matches = sorted(matches, key = lambda x:x.distance)

img4 = cv2.drawMatches(img1, keypoints_1, img3, keypoints_3, matches[:], img3, flags=2)
plt.imshow(img4),plt.show()

NameError: name 'bf' is not defined

In [6]:
def calculateMatches(des1,des2):
    matches = bf.knnMatch(des1,des2,k=2)
    topResults1 = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            topResults1.append([m])
            
    matches = bf.knnMatch(des2,des1,k=2)
    topResults2 = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            topResults2.append([m])
    
    topResults = []
    for match1 in topResults1:
        match1QueryIndex = match1[0].queryIdx
        match1TrainIndex = match1[0].trainIdx

        for match2 in topResults2:
            match2QueryIndex = match2[0].queryIdx
            match2TrainIndex = match2[0].trainIdx

            if (match1QueryIndex == match2TrainIndex) and (match1TrainIndex == match2QueryIndex):
                topResults.append(match1)
    return topResults

def calculateScore(matches,keypoint1,keypoint2):
    return 100 * (matches/min(keypoint1,keypoint2))

In [19]:

def getCarModel(carImagePath,path = './car_models_database_train/', modelsNames = []):
    
    #reading the image of the car ! 
    carImage = cv2.imread(carImagePath)
    carImage = cv2.cvtColor(carImage, cv2.COLOR_BGR2GRAY)
    #intilize the matching scores to models  
    matchingsScores = [0 for i in modelsNames]
    # creating SIFT
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints_car, descriptors_car = sift.detectAndCompute(carImage,None)
    
    for i in range(len(modelsNames)) :
        model_dir = Path(path + modelsNames[i])
        images = [path + modelsNames[i]+'/'+e.name for e in model_dir.iterdir() if e.is_file()]
        cpt = 0
        for img in images:
            cpt += 1 
            if(cpt == 4):
                break
            carImage2 = cv2.imread(img)
            carImage2 = cv2.cvtColor(carImage2, cv2.COLOR_BGR2GRAY)
            keypoints_car2, descriptors_car2 = sift.detectAndCompute(carImage2,None)
            matches = []
            matches = bf.match(descriptors_car,descriptors_car2)
            if(calculateScore(len(matches),len(keypoints_car),len(keypoints_car2)) > matchingsScores[i]):
                matchingsScores[i] = len(matches)
                
    indexModelMaxMatching = matchingsScores.index(max(matchingsScores))
    
    #return {"model": modelsNames[indexModelMaxMatching], "score":max(matchingsScores) }
    return modelsNames[indexModelMaxMatching]

In [20]:
train_dir = Path(trainPath)
trainImages = [trainPath+e.name for e in train_dir.iterdir() if e.is_file()]
idsImages   = [e.name.split(".",2)[0] for e in train_dir.iterdir() if e.is_file()]

df = pd.read_csv(annotation_file, sep=",",index_col=0)

In [21]:
mask = df['class'] != 'car'
df = df.mask(mask)
df.dropna(inplace=True)

In [22]:
df = df.loc[:,['im_name','models']]

In [23]:
df

,im_name,models
0,107347968.jpg,Audi S5 Convertible 2012
1,91160576.jpg,Audi 100 Wagon 1994
2,273809408.jpg,Cadillac SRX SUV 2012
3,579534848.jpg,Ford Freestar Minivan 2007
4,950599680.jpg,Rolls-Royce Phantom Sedan 2012
...,...,...
1304,630063104.jpg,Ford Fiesta Sedan 2012
1305,834142208.jpg,Land Rover LR2 SUV 2012
1306,812580864.jpg,Lamborghini Aventador Coupe 2012
1307,376569856.jpg,Chevrolet Express Van 2007


In [24]:
for i in range(len(df.index)):
    image = df.loc[i,'im_name']
    annotation_model = df.loc[i,'models'].replace(" ","")
    pathToImage = trainPath + image 
    model = getCarModel(carImagePath =pathToImage, path = './car_models_database_train/', modelsNames = carModels )
    print(model + ' ' + annotation_model) 

error: OpenCV(4.5.5) /io/opencv/modules/core/src/batch_distance.cpp:303: error: (-215:Assertion failed) K == 1 && update == 0 && mask.empty() in function 'batchDistance'


In [25]:
# Class labels in imagenet corresponding to cars
CAR_IDX = [656, 627, 817, 511, 468, 751, 705, 757, 717, 734, 654, 675, 864, 609, 436]

CAR_CLASSES = ['minivan', 'limousine', 'sports_car', 'convertible', 
               'cab', 'racer', 'passenger_car', 'recreational_vehicle', 
               'pickup', 'police_van', 'minibus', 'moving_van', 'tow_truck',
               'jeep', 'landrover', 'beach_wagon']


In [36]:
from tensorflow.keras.applications import ResNet50V2
import tensorflow as tf
model = ResNet50V2(weights='imagenet')

In [37]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input

filename = "../datasets/datasets_train/car_models_database_train/Acura ZDX Hatchback 2012_1.jpg"
image = tf.io.read_file(filename)
image = tf.image.decode_jpeg(image)
image = tf.cast(image, tf.float32)
image = tf.image.resize_with_crop_or_pad(image, target_height=224, target_width=224)
image = preprocess_input(image)
predictions = model.predict(image)

ValueError: in user code:

    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/opt/conda/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.8/site-packages/keras/engine/input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}"